# SNIa Multi-band Lightcurve
By Melissa Graham (mlg3k@uw.edu)

#### Credit
This NB draws on code from the Rubin DP0 tutorial notebook `09_Single_Star_Lightcurve` by Jeff Carlin.

#### Objectives
Find a SNIa that returns a decent looking multi-band light curve.
 - Query the truth match catalog to find SNeIa.
 - Query the Butler to get Source measurements for a SNIa.
 - Plot the multi-band light curve for a SNIa.
 
#### Stretch Goals
Left for other notebooks.
 - Retrieve magnitude errors and plot them too.
 - Find an even better-sampled SNIa.
 - Fit a SNIa template lightcurve.
 - Identify and characterize the host galaxy.


## 0. Setup

In [ ]:
### Rubin-specific packages

import lsst.daf.butler as dafButler
import lsst.geom as geom
import lsst.sphgeom as sphgeom
import lsst.daf.base as dafBase

### If you want to use the TAP Service in Section 2, uncomment.
from rubin_jupyter_utils.lab.notebook import get_tap_service
service = get_tap_service()

### General python / astronomy packages

import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
from astropy.table import Table
from astropy.io import fits
from astropy.timeseries import LombScargle

import time

### 0.1 Instatiate Butler

In [ ]:
repo = 's3://butler-us-central1-dp01'
collection = "2.2i/runs/DP0.1"
butler = dafButler.Butler(repo, collections=collection)
registry = butler.registry

### 0.2 TAP Query for SNeIa

Query the truth match catalog. Use central coordinates of DC2. Use a 5 degree radius. Use is_variable = 1 (true) to only return variables. Use truth_type = 3 to only return Type Ia supernovae. Use is_unique_truth_entry = 'true' to ensure good truth-table matches only. Use redshift < 0.3 to be more likely to get a full light curve with lots of data points.

In [ ]:
results = service.search("SELECT ra, dec "\
                         "FROM dp01_dc2_catalogs.truth_match "\
                         "WHERE CONTAINS(POINT('ICRS', ra, dec), "\
                         "CIRCLE('ICRS', 62.0, -37.0, 5.0)) = 1 "\
                         "AND is_variable = 1 AND truth_type = 3 AND is_unique_truth_entry = 'true' AND redshift < 0.3 ",\
                         maxrec=10000)

#### Show the TAP query results

In [ ]:
data = results.to_table().to_pandas()
data

## 1. Get light curve data for one SNIa

In [ ]:
### At first, I tried making just the r-band the light curve for each of these in turn
###  Only two of the following looked like useful SNIa light curves
# i = 0   # r-band did not look SNIa-like
# i = 1   # r-band results had no mjd, no mag
# i = 2   # r-band looks SNIa-like, if a bit undersampled
# i = 3   # r-band results had no mjd, no mag
# i = 4   # r-band detections on decline only
i = 5   # r-band looks SNIa-like
# i = 6   # r-band only three epochs
# i = 7   # r-band only two epochs

print( data['ra'][i] * u.deg, data['dec'][i] * u.deg )

targ_coord = SkyCoord(ra=data['ra'][i] * u.deg, dec=data['dec'][i] * u.deg)

### 1.1 Define the HTM ID for the Butler
Use these coordinates to define the HTM ID spatial search region to pass to the Butler's `queryDatasets` function.

In [ ]:
pixelization = sphgeom.HtmPixelization(15)
htm_id = pixelization.index(sphgeom.UnitVector3d(sphgeom.LonLat.fromDegrees(
                            targ_coord.ra.value, targ_coord.dec.value)))

# Obtain and print the scale to provide a sense of the size of the
#   sky pixelization being used
scale = pixelization.triangle(htm_id).getBoundingCircle().getOpeningAngle().asDegrees() * 3600
print(f'HTM ID={htm_id} at level={pixelization.getLevel()} is a ~{scale:0.2}" triangle.')

### 1.2 Query the Butler
Query the `src` datasets (i.e., measurements from each processed visit image) for this HTM pixel via the Butler.

In [ ]:
### To retrieve only r-band
# datasetRefs = registry.queryDatasets("src", htm20=htm_id,
#                                      collections=collection,
#                                      where="band in ('r')")

### To retrieve all filters
datasetRefs = registry.queryDatasets("src", htm20=htm_id,
                                     collections=collection)

In [ ]:
datasetRefs

In [ ]:
refs = list(datasetRefs)
totalNrefs = len(refs)

print(totalNrefs, ' catalogs matching the requested position.')

**WARNING** The following cell can take a few minutes to execute.

In [ ]:
t0 = time.time()
print(t0)

# To retrieve data for all `refs`, set N_refs to be equal to totalNrefs (or to a large value like 1000).
# Alternatively to test the retrieval of a few `refs`, set smaller (e.g., 10).
Nrefs = totalNrefs
# Nrefs = 5

# Instantiate empty lists
ra_arr = []
dec_arr = []
sep_arr = []
mag_arr = []
visit_arr = []
detector_arr = []
mjd_arr = []
band_arr = []

# Loop over all refs
for i, d in enumerate(refs):
    t1 = time.time()

    if i <= Nrefs:

        # Use the butler to get all sources for this datasetRef's dataId
        did = d.dataId
        src = butler.get('src', dataId=did)

        # Get the separation of all sources from the target
        src_coords = SkyCoord(ra=src['coord_ra'] * u.rad,
                              dec=src['coord_dec'] * u.rad)
        sep = src_coords.separation(targ_coord)

        # If the nearest source is within 1.5", append source quantities to python lists
        if np.min(sep.arcsecond) < 1.5:
            sx = np.argmin(sep.arcsecond)

            # Append RA, Dec, and separation
            ra_arr.append(src['coord_ra'][sx])
            dec_arr.append(src['coord_dec'][sx])
            sep_arr.append(sep[sx].arcsecond)

            # Append r-band magnitude (AB mag from the calibrated flux in nJy).
            # The calibrated flux in nJy is base_PsfFlux_instFlux times base_localPhotoCalib
            mag_arr.append(-2.5 * np.log10(src['base_PsfFlux_instFlux'][sx] * src['base_localPhotoCalib'][sx]) + 31.4)

            # Append visit, detector, and band information
            visit_arr.append(did['visit'])
            detector_arr.append(did['detector'])
            band_arr.append(did['band'])

            # Append MJD (from this src's associated image's header)
            visit_info = butler.get('calexp.visitInfo', dataId=did)
            mjd_arr.append(visit_info.getDate().get(dafBase.DateTime.MJD))

            del sx

        t2 = time.time()
        if i == 0:
            print('Each data point takes: ', t2 - t1, ' seconds to retrieve.')
            print('Retrieving ', Nrefs, ' data points will take ',
                  (t2 - t1) * Nrefs / 60.0, ' minutes.')

        del did, src, src_coords, sep
        del t1, t2

print('Total run-time was: ', (time.time() - t0) / 60.0, ' minutes.')
del t0

### 1.3 Convert to numpy arrays

In [ ]:
### Convert to pandas table via an astropy table
###  Including all columns for now just as a demo
tab_timeseries = Table([mjd_arr, ra_arr, dec_arr, sep_arr, mag_arr,
                        visit_arr, detector_arr, band_arr],
                       names=['mjd', 'ra', 'dec', 'separation', 'mag',
                              'visit', 'detector', 'band']).to_pandas()

In [ ]:
### Convert to numpy arrays
###  Just taking the columns I will want to plot
tab_mjds = np.asarray( tab_timeseries['mjd'], dtype='float' )
tab_mags = np.asarray( tab_timeseries['mag'], dtype='float' )
tab_bands = np.asarray( tab_timeseries['band'], dtype='str' )

## 2.0 Plot the multi-band lightcurve

In [ ]:
use_point_colors = ['darkviolet','darkgreen','darkorange','red','brown','black']

fig = plt.figure(figsize=(12, 7))
plt.rcParams.update({'font.size': 22})

for f,fil in enumerate( ['u','g','r','i','z','y'] ):
    tx = np.where( tab_bands == fil )[0]
    if len(tx) > 0:
        plt.plot(tab_mjds[tx], tab_mags[tx], 'o', ms=10, alpha=0.5, mew=0, color=use_point_colors[f], label=fil)

plt.xlabel('Modified Julian Date',fontsize=22)
plt.ylabel('Apparent Magnitude')
plt.gca().invert_yaxis()
plt.minorticks_on()
plt.xlim([59820,60000])
plt.legend(loc='upper right')
plt.show()